In [53]:
import numpy as np
import random 

from scripts import viterby


def expirement(N = 1000, n = 1000, mode = True):
    
    if mode:
        s = 0
        for i in range(N):
            d = distibution(6, 1000)/1000
            trn = [[0.95, 0.05],
                   [0.1, 0.9]]
            symbols = [i+1 for i in range(6)]
            dist = dict(zip(symbols,d))
            ems = [dist,dist]
            s+= test(trn, ems, n)
        return s/N
    
    s = 0
    for i in range(N):
        trn = [[0.95, 0.05],
                [0.1, 0.9]]
        symbols = [i+1 for i in range(6)]
        a = np.random.randint(1000, 100000)
        d = [1/a, 1/a,1/a, 1/a,1/a, 1-5/a]
        random.shuffle(d)
        ems = [
            {1:1/6,2:1/6,3:1/6,4:1/6,5:1/6,6:1/6},
            dict(zip(symbols,d))]
        s+= test(trn, ems, n)
    return s/N

def generate(trn, ems, n = 10000):
    states = [np.random.randint(2)]
    for i in range(1,n):
        state = states[-1]
        prob = trn[state][1]
        states.append(np.random.binomial(1, prob, size=None))
    symbols = []
    all_symbols = [i+1 for i in range(len(ems[0]))]
    for i in range(n):
        symbol = np.random.choice(all_symbols,1,  ems[states[i]])
        symbols.append(symbol[0])
    
    return states, symbols


def test(trn, ems, n):
    states, symbols = generate(trn, ems, n)
    prediction = viterby(
        observations=symbols, 
        init_prob=[1/2, 1/2],
        transition_prob=trn,
        emision_prob=ems
    )
    accuracy = (np.array(prediction)==np.array(states)).sum()/n
    return accuracy

def distibution(n, total):
    """Return a randomly chosen list of n positive integers summing to total.
    Each such list is equally likely to occur."""

    dividers = sorted(random.sample(range(1, total), n - 1))
    return np.array([a - b for a, b in zip(dividers + [total], [0] + dividers)])


# Описание экспериментов

Цель экспериментов - проверить точность алгоритма Витерби при разных матрицах эмиссий, но с одинаковыми матрицам переходов. В качестве матрицы переходов зафиксировал:
```
[[0.95, 0.05],
 [0.1,  0.9 ]]
```
Рассмотрел два класса матриц эмиссий:
1. С нулевой дивергенцией, на каждой итерации эксперимента генирировал одинаковое для обоих состояний произвольное распределение из 6 символов.
2. С большой дивергенцией для:
    
 - первого состояния распределение на каждой итерации равно `[1/6,1/6,1/6,1/6,1/6,1/6]`
 - второго состояяния распределения имеет очень большую вероятность в одном из символов и одинаковую в остальных. Например,`[495/500,1/500,1/500,1/500,1/500, 1/500]`

Итерация эксперимента:
1. Генерируются параметры скрытой марковской цепи в соответствии с правилами выше для эмиссий с большой или малой дивергенцией
2. Генерируется последовательность скрытых состояний и символов длиной `100`
3. Применяется алгоритм Витерби и считаем процент верно предсказанных скрытых состояний

В каждом экперименте `100 000` итераций, полсе того, как все они сработали, считаем и выводим среднюю точность алгоритма Витерби по всем итерациями.


In [107]:
# Нулевая дивергенция
expirement(100000,100)

0.6553180999999957

In [108]:
# Большая дивергенция
expirement(100000,100, False)

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


0.6531375000000066